In [1]:
!git clone https://github.com/Hotsnown/seminaire-bordeaux-2022.git seminaire &> /dev/null
%pip install nbautoeval &> /dev/null
from seminaire.evaluation.jour4.textclassification.textclassificaton import 

SyntaxError: invalid syntax (3958868141.py, line 3)

Classification en langage naturel
Vous avez fait un si bon travail pour le restaurant DeFalco dans l'exercice précédent que le chef vous a engagé pour un nouveau projet.

Le menu du restaurant comprend une adresse e-mail où les visiteurs peuvent donner leur avis sur la nourriture.

Le chef veut que vous créiez un outil qui lui envoie automatiquement toutes les critiques négatives afin qu'il puisse les corriger, tout en envoyant automatiquement toutes les critiques positives au propriétaire, afin que le chef puisse demander une augmentation.

Vous allez d'abord construire un modèle pour distinguer les avis positifs des avis négatifs en utilisant les avis Yelp, car ces avis incluent une note avec chaque avis. Vos données sont constituées du corps du texte de chaque avis ainsi que de l'évaluation par étoiles. Les évaluations avec 1 à 2 étoiles sont considérées comme "négatives", et celles avec 4 à 5 étoiles sont "positives". Les évaluations avec 3 étoiles sont "neutres" et ont été supprimées des données.

#### Étape 1 : Évaluer l'approche

Y a-t-il un aspect de cette approche qui vous préoccupe ? Après avoir réfléchi à la question, exécutez la fonction ci-dessous pour voir un point de vue.


In [ ]:
# Check your answer (Run this code cell to receive credit!)
#step_1.solution()
step_1.check()

Y a-t-il un aspect de cette approche qui vous préoccupe ? Après avoir réfléchi à la question, exécutez la fonction ci-dessous pour voir un point de vue.


Toute façon de mettre en place un problème de ML aura de multiples forces et faiblesses. Il se peut donc que vous ayez pensé à d'autres problèmes que ceux énumérés ici.

La force de cette approche est qu'elle vous permet de distinguer les courriels positifs des courriels négatifs, même si vous ne disposez pas de courriels historiques que vous avez étiquetés comme positifs ou négatifs.

La faiblesse de cette approche est que les e-mails peuvent être systématiquement différents des avis Yelp d'une manière qui rend votre modèle moins précis. Par exemple, les clients peuvent généralement utiliser des mots différents ou de l'argot dans les e-mails, et le modèle basé sur les avis Yelp n'aura pas vu ces mots.

Si vous voulez voir à quel point ce problème est sérieux, vous pouvez comparer la fréquence des mots entre les deux sources. Dans la pratique, la lecture manuelle de quelques courriels de chaque source peut suffire à déterminer s'il s'agit d'un problème grave.

Si vous vouliez faire quelque chose de plus sophistiqué, vous pourriez créer un ensemble de données contenant à la fois des avis Yelp et des courriels et voir si un modèle peut déterminer la source des avis à partir du contenu textuel. Idéalement, vous aimeriez constater que ce modèle n'a pas donné de bons résultats, car cela signifierait que vos sources de données sont similaires. Cette approche semble inutilement complexe ici.

#### Étape 2 : Examen des données et création du modèle


Pour avancer dans votre plan, vous devez charger les données. Voici un code de base pour charger les données et les diviser en un ensemble de formation et de validation. Exécutez ce code.

In [ ]:
def load_data(csv_file, split=0.9):
    data = pd.read_csv(csv_file)
    
    # Shuffle data
    train_data = data.sample(frac=1, random_state=7)
    
    texts = train_data.text.values
    labels = [{"POSITIVE": bool(y), "NEGATIVE": not bool(y)}
              for y in train_data.sentiment.values]
    split = int(len(train_data) * split)
    
    train_labels = [{"cats": labels} for labels in labels[:split]]
    val_labels = [{"cats": labels} for labels in labels[split:]]
    
    return texts[:split], train_labels, texts[split:], val_labels

train_texts, train_labels, val_texts, val_labels = load_data('../input/nlp-course/yelp_ratings.csv')

Vous allez utiliser ces données d'entraînement pour construire un modèle. Le code pour construire le modèle est le même que celui que vous avez vu dans le tutoriel. Il est donc copié ci-dessous pour vous.

Mais comme vos données sont différentes, il y a deux lignes dans la cellule de code de modélisation que vous devrez modifier. Pouvez-vous trouver ce qu'elles sont ?

Tout d'abord, exécutez la cellule ci-dessous pour examiner quelques éléments de vos données d'entraînement.

In [ ]:
print('Texts from training data\n------')
print(train_texts[:2])
print('\nLabels from training data\n------')
print(train_labels[:2])

In [ ]:
import spacy

# Create an empty model
nlp = spacy.blank("en")

# Create the TextCategorizer with exclusive classes and "bow" architecture
textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})

# Add the TextCategorizer to the empty model
nlp.add_pipe(textcat)

# Add labels to text classifier
textcat.add_label("NEGATIVE")
textcat.add_label("POSITIVE")

# Check your answer
step_2.check()

#### Étape 3 : Entraînement de la fonction


Implémenter une fonction train qui met à jour un modèle avec des données d'entraînement. La plupart de ces fonctions sont des fonctions générales de gestion des données, que nous avons remplies pour vous. Ajoutez simplement la ligne de code nécessaire pour mettre à jour votre modèle.



In [ ]:
from spacy.util import minibatch
import random

def train(model, train_data, optimizer):
    losses = {}
    random.seed(1)
    random.shuffle(train_data)
    
    batches = minibatch(train_data, size=8)
    for batch in batches:
        # train_data is a list of tuples [(text0, label0), (text1, label1), ...]
        # Split batch into texts and labels
        texts, labels = zip(*batch)
        
        # Update model with texts and labels
        model.update(texts, labels, sgd=optimizer, losses=losses)
        
    return losses

# Check your answer
step_3.check()

In [ ]:
# Lines below will give you a hint or solution code
#step_3.hint()
#step_3.solution()

In [ ]:
# Fix seed for reproducibility
spacy.util.fix_random_seed(1)
random.seed(1)

# This may take a while to run!
optimizer = nlp.begin_training()
train_data = list(zip(train_texts, train_labels))
losses = train(nlp, train_data, optimizer)
print(losses['textcat'])

Nous pouvons essayer ce modèle légèrement entraîné sur un texte d'exemple et examiner les probabilités attribuées à chaque étiquette.

In [ ]:
text = "This tea cup was full of holes. Do not recommend."
doc = nlp(text)
print(doc.cats)

Ces probabilités semblent raisonnables. Maintenant, vous devez les transformer en une prédiction réelle.

#### Étape 4 : Faire des prédictions

Implémenter une fonction predict qui prédit le sentiment des exemples de textes.

D'abord, tokeniser les textes en utilisant nlp.tokenizer().<br/>
Ensuite, passez ces documents au TextCategorizer que vous pouvez obtenir de nlp.get_pipe().<br/>
Utilisez la méthode textcat.predict() pour obtenir des scores pour chaque document, puis choisissez la classe avec le score le plus élevé (probabilité) comme classe prédite.<br/>

In [ ]:
def predict(nlp, texts): 
    # Use the model's tokenizer to tokenize each input text
    docs = [nlp.tokenizer(text) for text in texts]
    
    # Use textcat to get the scores for each doc
    textcat = nlp.get_pipe('textcat')
    scores, _ = textcat.predict(docs)
    
    # From the scores, find the class with the highest score/probability
    predicted_class = scores.argmax(axis=1)
    
    return predicted_class

# Check your answer
step_4.check()

In [ ]:
texts = val_texts[34:38]
predictions = predict(nlp, texts)

for p, t in zip(predictions, texts):
    print(f"{textcat.labels[p]}: {t} \n")

Il semble que votre modèle fonctionne bien après avoir parcouru les données une seule fois. Cependant, vous devez calculer une métrique pour la performance du modèle sur les données de validation retenues.

#### Étape 5 : Évaluation du modèle


Implémentez une fonction qui évalue un modèle TextCategorizer. Cette fonction d'évaluation prend un modèle avec des textes et des étiquettes. Elle renvoie la précision du modèle, qui est le nombre de prédictions correctes divisé par toutes les prédictions.

Tout d'abord, utilisez la méthode predict que vous avez écrite précédemment pour obtenir la classe prédite pour chaque texte dans les textes. Ensuite, trouvez où les étiquettes prédites correspondent aux véritables étiquettes "gold-standard" et calculez la précision.

In [ ]:
predict(nlp, texts)


In [ ]:
def evaluate(model, texts, labels):
    """ Returns the accuracy of a TextCategorizer model. 
    
        Arguments
        ---------
        model: ScaPy model with a TextCategorizer
        texts: Text samples, from load_data function
        labels: True labels, from load_data function
    
    """
    # Get predictions from textcat model (using your predict method)
    predicted_class = predict(model, texts)
    
    # From labels, get the true class as a list of integers (POSITIVE -> 1, NEGATIVE -> 0)
    true_class = [int(label['cats']['POSITIVE']) for label in labels]
    
    # A boolean or int array indicating correct predictions
    correct_predictions = true_class == predicted_class
    
    # The accuracy, number of correct predictions divided by all predictions
    accuracy = correct_predictions.mean()
    
    return accuracy

step_5.check()

In [ ]:
accuracy = evaluate(nlp, val_texts, val_labels)
print(f"Accuracy: {accuracy:.4f}")

Avec les fonctions implémentées, vous pouvez former et évaluer dans une boucle.



In [ ]:
# This may take a while to run!
n_iters = 5
for i in range(n_iters):
    losses = train(nlp, train_data, optimizer)
    accuracy = evaluate(nlp, val_texts, val_labels)
    print(f"Loss: {losses['textcat']:.3f} \t Accuracy: {accuracy:.3f}")

#### Étape 6 : Continuer à s'améliorer

Vous avez construit les composants nécessaires pour entraîner un classificateur de texte avec spaCy. Que pourriez-vous faire de plus pour optimiser le modèle ?

Exécutez la ligne suivante pour vérifier votre réponse.

1. Jour 1
    * Variables
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.1%20helloworld.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.2%20%C3%A9viter%20les%20errreurs%20de%20nommage.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/1.%20variables/1.3%20mini-project.ipynb#scrollTo=RPB2xCMdA6lV)
    * Strings
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.1%20concat.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.2%20string_methods.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/2.%20strings/2.3%20mini-project.ipynb)
    * Opérations
        * [exercice1](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.1%20math.ipynb)
        * [exercice2](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.2%20bool%C3%A9en.ipynb)
        * [mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour1/3.%20operations/3.3%20mini-project.ipynb)

2. Jour 2
    * Listes
        * [Définition](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.1%20d%C3%A9finition%20liste.ipynb?hl=fr)
        * [Strings et listes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.2%20String%20as%20list%20of%20characters.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.1%20Listes/2.1.3%20mini-project.ipynb?hl=fr)
    * Fonctions
        * [Définition I](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.1%20d%C3%A9finition%20fonctions.ipynb?hl=fr)
        * [Définition II](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.2%20scope%20et%20fonctions%20imbriqu%C3%A9es.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.2%20Fonctions/2.2.3%20mini-project.ipynb?hl=fr) 
    * Librairies
        * [Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.1%20Pandas.ipynb?hl=fr)
        * [Matplotlib](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.2%20Matplotlib.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour2/2.3%20Librairies/2.3.3%20mini-project.ipynb?hl=fr) 
3. Jour 3
    * Introduction à la NLP
        * [Charger des un corpus](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.1%20Accessing%20Text.ipynb?hl=fr)
        * [Traitement de texte dans Pandas](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.2%20Working%20with%20text%20data%20in%20pandas.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.1%20what%20is%20nlp/3.1.3%20mini-project.ipynb?hl=fr)
    * Segmentation
        * [Segmentation de tokens](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.1%20Token%20segmentation.ipynb?hl=fr)
        * [Segmentation de phrase](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.2%20Sentence%20segmentation.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.2%20Segmentation/3.2.3%20mini-project.ipynb?hl=fr)
    * Nettoyage de texte
        * [Stopwords](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.1%20stopwords.ipynb?hl=fr)
        * [Normalisation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.2%20Normalizing%20Text.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour3/3.3%20text%20cleaning.ipynb/3.3.3%20mini-project.ipynb?hl=fr)
4. Jour 4
    * Apprentissage supervisé
        * [Régression linéaire](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.1%20linear%20regression.ipynb?hl=fr)
        * [Evaluation](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.2%20evaluale.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.1%20supervised%20learning/4.3.3%20mini-project.ipynb?hl=fr)
    * Pré-traitement de texte
        * [Featurization de textes](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.1%20text%20featurization.ipynb?hl=fr)
        * [Featurization de labels](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.2%20label%20featurization.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.2%20text%20preprocessing/4.2.3%20mini-project.ipynb?hl=fr)
    * Classification de texte
        * [EDA](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Apprentissage supervisé textuel](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.1%20EDA.ipynb?hl=fr)
        * [Mini-projet](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour4/4.3%20text%20classification/4.1.3%20mini-project.ipynb?hl=fr)
5. Jour 5
    * [Projet final](https://colab.research.google.com/github/Hotsnown/seminaire-bordeaux-2022/blob/master/exercices/jour5/final-project.ipynb?hl=fr)